In [1]:
!pip install transformers torch nltk rouge-score

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━

In [4]:
from transformers import BartForConditionalGeneration, BartTokenizer
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [5]:
# Load the pre-trained BART model and tokenizer

model_name = 'facebook/bart-large-cnn'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [10]:
def preprocess_text(text):
    inputs = tokenizer.batch_encode_plus([text], max_length=1024, return_tensors='pt', truncation=True)
    return inputs

def summarize_text(text):
    inputs = preprocess_text(text)
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=500, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def evaluate_summary(reference, summary):
    bleu_score = sentence_bleu([reference.split()], summary.split())
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(reference, summary)
    return bleu_score, rouge_scores




In [18]:
# Example usage
text = """
Prime Minister Narendra Modi interacted with the Indian Cricket team at 7, Lok Kalyan Marg yesterday,
celebrating their recent victory in the ICC Men's T20 World Cup 2024. During the interaction,
PM Modi expressed his heartfelt congratulations and admiration for the team's outstanding performance.
He said, "It is a matter of joy for us that you have filled our country with enthusiasm and celebration
and you have fulfilled all the hopes and expectations of our countrymen. My heartiest congratulations to you...
" The Prime Minister acknowledged the hard work and dedication of the players, highlighting how their victory
has brought immense pride and happiness to the nation. He commended the team for their exemplary
sportsmanship and unity, which played a crucial role in their success. The interaction was filled with
camaraderie and mutual respect, reflecting the nation's gratitude towards its cricket heroes.
The event at Lok Kalyan Marg was not just a celebration of the World Cup win but also a moment of national pride
 and inspiration, as PM Modi encouraged the team to continue striving for excellence and to keep making the
country proud on the global stage.
"""

reference_summary = " PM Modi expressed his heartfelt congratulations and admiration for the team's outstanding performance. ."


In [19]:
summary = summarize_text(text)
print("Generated Summary:", summary)


Generated Summary: PM Modi expressed his heartfelt congratulations and admiration for the team's outstanding performance. The event at Lok Kalyan Marg was not just a celebration of the World Cup win but also a moment of national pride and inspiration. PM Modi encouraged the team to continue striving for excellence and to keep making the country proud.


In [20]:
bleu_score, rouge_scores = evaluate_summary(reference_summary, summary)
print("BLEU Score:", bleu_score)
print("ROUGE Scores:", rouge_scores)


BLEU Score: 0.21397863129191896
ROUGE Scores: {'rouge1': Score(precision=0.25, recall=1.0, fmeasure=0.4), 'rougeL': Score(precision=0.25, recall=1.0, fmeasure=0.4)}
